# Aula 8: Estruturas de Dados Avançadas

Na Unidade 3, abordamos estruturas que suportam algoritmos de alta performance em cenários de grande volume de dados.

## Objetivos de Aprendizagem

Ao final desta aula, você deverá ser capaz de:
1. Entender a importância de estruturas balanceadas para operações em tempo logarítmico.
2. Definir fatores de balanceamento e implementar rotações em árvores AVL.
3. Inserir elementos em AVL com rebalanceamento automático.
4. Compreender e usar Heaps (min-heap / max-heap) para filas de prioridade.
5. Ter uma visão geral de B-Trees e sua aplicação em bancos de dados.

## 1. Árvores AVL – Revisão de Rotações

### Conceitos de AVL e Tipos de Rotações
### 1.1 Motivação

- Em uma **Árvore Binária de Busca (BST)** simples, inserções/destruições sequenciais podem degenerar para uma lista ligada ($(O(n))$).  
- **AVL** (Adelson-Velskii & Landis) garante que, para todo nó,  
  $$
    bigl|\text{altura(esquerda)} - \text{altura(direita)}\bigr| \le 1.
  $$  
- Isso mantém a **altura** da árvore em $(O(\log n))$.

### 1.2 Fator de Balanceamento (FB)

Para cada nó $(x)$:
$$
  \text{FB}(x) = \text{altura}(x.\text{esq}) \;-\; \text{altura}(x.\text{dir}),
$$
deve satisfazer $(\text{FB}\in\{-1,0,+1\})$.

### 1.3 Tipos de Rotação

Quando $(\lvert\text{FB}\rvert>1)$, aplicamos:

| Caso        | Condição                                         | Rotações                       |
|-------------|--------------------------------------------------|--------------------------------|
| **LL**      | FB > 1 e FB(esq) ≥ 0                             | Rotação **Direita**            |
| **RR**      | FB < −1 e FB(dir) ≤ 0                            | Rotação **Esquerda**           |
| **LR**      | FB > 1 e FB(esq) < 0                              | Rotação **Esquerda** em filho + Direita no nó |
| **RL**      | FB < −1 e FB(dir) > 0                             | Rotação **Direita** em filho + Esquerda no nó |

### 1.3 Exemplo Animado

W3Schools - AVL Trees
https://www.w3schools.com/dsa/dsa_data_avltrees.php

### 1.4 Pseudocódigo de Inserção em AVL

```python
INSERT_AVL(node, key):
  if node == null:
    return NovoNo(key)

  if key < node.key:
    node.esq = INSERT_AVL(node.esq, key)
  else:
    node.dir = INSERT_AVL(node.dir, key)

  # Atualiza altura
  node.altura = 1 + max(altura(node.esq), altura(node.dir))

  fb = altura(node.esq) - altura(node.dir)

  # LL
  if fb > 1 and key < node.esq.key:
    return ROTATE_RIGHT(node)

  # RR
  if fb < -1 and key > node.dir.key:
    return ROTATE_LEFT(node)

  # LR
  if fb > 1 and key > node.esq.key:
    node.esq = ROTATE_LEFT(node.esq)
    return ROTATE_RIGHT(node)

  # RL
  if fb < -1 and key < node.dir.key:
    node.dir = ROTATE_RIGHT(node.dir)
    return ROTATE_LEFT(node)

  return node

In [ ]:
class AVLNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.height = 1  # nó inicial como folha

def height(node):
    return node.height if node else 0

def update_height(node):
    node.height = 1 + max(height(node.left), height(node.right))

def rotate_right(y):
    x = y.left
    T2 = x.right
    x.right = y
    y.left = T2
    update_height(y)
    update_height(x)
    return x

def rotate_left(x):
    y = x.right
    T2 = y.left
    y.left = x
    x.right = T2
    update_height(x)
    update_height(y)
    return y


## 2. Inserção em AVL com Rebalanceamento

Ao inserir um novo nó, atualizamos alturas e checamos FB de cada ancestro, aplicando rotações se necessário.

In [ ]:
def insert_avl(node, key):
    if not node: return AVLNode(key)
    if key < node.key:
        node.left = insert_avl(node.left, key)
    else:
        node.right = insert_avl(node.right, key)
    update_height(node)
    fb = height(node.left) - height(node.right)
    if fb > 1 and key < node.left.key:
        return rotate_right(node)  # LL
    if fb < -1 and key > node.right.key:
        return rotate_left(node)   # RR
    if fb > 1 and key > node.left.key:
        node.left = rotate_left(node.left)  # LR
        return rotate_right(node)
    if fb < -1 and key < node.right.key:
        node.right = rotate_right(node.right)  # RL
        return rotate_left(node)
    return node

# Teste de inserção
root = None
for v in [10,20,30,40,50,25]:
    root = insert_avl(root, v)
print('AVL inserido com rebalanceamento concluído')

## 3. Heaps e B-Trees

### 3.1 Heaps (Filas de prioridades)
- **Min-Heap**: raiz contém o menor elemento.
- **Max-Heap**: raiz contém o maior elemento (implementado com valores negativos em min-heap).
- Operações: `push`, `pop` em $O(\log n)$; acesso ao topo em $O(1)$.

| Operação       | Descrição                              | Complexidade                     |
| -------------- | -------------------------------------- | -------------------------------- |
| **peek/pop**   | Retorna e/ou remove o elemento do topo | $O(\log n)$ (pop), $O(1)$ (peek) |
| **push**       | Insere novo elemento                   | $O(\log n)$                      |
| **build-heap** | Constrói heap a partir de array        | $O(n)$                           |


In [ ]:
import heapq

# Min-Heap
nums = [5,1,7,3]
heap = []
for x in nums:
    heapq.heappush(heap, x)
print(heap[0])          # topo = 1
print(heapq.heappop(heap))  # remove 1

# Max-Heap via negativos
max_heap = []
for x in nums:
    heapq.heappush(max_heap, -x)
print(-max_heap[0])     # topo = 7

### 3.2 B-Trees – Visão Geral
- **Árvore multiway**: cada nó pode ter até `m` filhos, otimizada para acesso em disco/SSD.
- **Propriedades**: todos os nós folhas no mesmo nível; número de chaves em cada nó entre `⌈m/2⌉-1` e `m-1`.
- **Uso**: índices de bancos de dados e sistemas de arquivos, minimizando leituras de disco.


**Pseudocódigo**: Inserção em B-Tree  order `m`:
````python
B_TREE_INSERT(T, k):
  r = T.root
  if r.n == 2t – 1:           # nó cheio (t = ⌈m/2⌉)
    s = NovoNo()
    T.root = s
    s.folha = False
    s.n = 0
    s.filho[0] = r
    SPLIT_CHILD(s, 0)
    INSERT_NONFULL(s, k)
  else:
    INSERT_NONFULL(r, k)

INSERT_NONFULL(x, k):
  i = x.n – 1
  if x.folha:
    while i ≥ 0 and k < x.chave[i]:
      x.chave[i+1] = x.chave[i]
      i--
    x.chave[i+1] = k
    x.n++
  else:
    while i ≥ 0 and k < x.chave[i]:
      i--
    i++
    if x.filho[i].n == 2t – 1:
      SPLIT_CHILD(x, i)
      if k > x.chave[i]:
        i++
    INSERT_NONFULL(x.filho[i], k)
    ````


### 4. Quando usar cada estrutura

| Estrutura    | Operações típicas                             | Melhor para…                         |
| ------------ | --------------------------------------------- | ------------------------------------ |
| **AVL Tree** | Inserção/remoção/busca $O(\log n)$            | BST com acesso dinâmico, memória RAM |
| **Heap**     | Prioridade: push/pop $O(\log n)$, topo $O(1)$ | Filas de prioridade                  |
| **B-Tree**   | Inserção/busca em disco $O(\log n)$           | Índices de BD, sistemas de arquivos  |


### 5. Dicas e armadilhas

Avalie carga de trabalho e padrão de acesso antes de escolher.

Em AVL, lembre-se de atualizar alturas após cada rotação.

Em Heaps, use heapq em Python ou faça sua própria implementação em C/C++ para desempenho ótimo.

Em B-Trees, entenda parâmetros 
𝑚
m e tamanho de nó em disco para máxima eficiência I/O.